<a href="https://colab.research.google.com/github/Aravind-kota/aravind/blob/main/Acko_web_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!apt-get update
!apt-get install -y wget
!wget -q -O - https://dl.google.com/linux/linux_signing_key.pub | apt-key add -
!echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google-chrome.list
!apt-get update
!apt-get install -y google-chrome-stable
!apt-get install -y chromium-chromedriver
!pip install selenium

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,565 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,858 kB]
Hit:12 https://dl.google.com/linux/chrome/deb stable InRelease
Fetched 12.4 MB in 1s (10.4 MB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.e

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd

In [13]:
from bs4 import BeautifulSoup
import requests

web_link = "https://ackodrive.com/collection/honda+cars/"
response = requests.get(web_link)

soup = BeautifulSoup(response.content,'html.parser')

In [14]:
raw_cars = soup.find_all(class_="BuyCarCard_carName__SAJVh")
len(raw_cars)

all_cars = []
for name in raw_cars:
    all_cars.append(name.text.strip())
cars_to_check = []
car_names = soup.find_all(class_="BuyCarCard_card__AsGXF")
for car_name in car_names:
        if "Express Delivery" in car_name.text.strip():
                for car in all_cars:
                        if car in car_name.text.strip():
                                cars_to_check.append(car)

In [15]:
cars_to_check

['Honda City', 'Honda Elevate', 'Honda Amaze']

In [16]:
def get_complete_variants(car):
    driver = None
    try:
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-gpu')
        options.add_argument('--window-size=1920,1080')
        options.add_argument('--disable-blink-features=AutomationControlled')

        driver = webdriver.Chrome(options=options)
        driver.set_page_load_timeout(30)

        car_domain = car.lower().replace(" ", "-")
        url = f"https://ackodrive.com/cars/{car_domain}/variants/"

        driver.get(url)
        time.sleep(3)

        # Click view all variants button
        try:
            button = driver.find_element(By.CLASS_NAME, "VariantWisePrice_viewMoreLink__CkpjG")
            driver.execute_script("arguments[0].click();", button)
            time.sleep(2)
        except:
            pass

        # Get unique variant names
        variant_cards = driver.find_elements(By.XPATH, "//div[contains(@class, 'CarVariantCard')]")

        variants_to_process = []
        seen_names = set()

        for card in variant_cards:
            try:
                variant_name = card.find_element(By.XPATH, ".//h2[contains(@class, 'CarVariantCard_variantName')]").text.strip()
                has_badge = len(card.find_elements(By.CLASS_NAME, "CarVariantCard_discontinuedBadge__bqsA8")) > 0

                if variant_name and not has_badge and variant_name not in seen_names:
                    variants_to_process.append(variant_name)
                    seen_names.add(variant_name)
            except:
                continue

        variants_data = []

        # Process each variant
        for variant_name in variants_to_process:
            try:
                # Re-fetch cards to avoid stale elements
                variant_cards = driver.find_elements(By.XPATH, "//div[contains(@class, 'CarVariantCard')]")

                # Find the matching card
                target_card = None
                for card in variant_cards:
                    try:
                        card_name = card.find_element(By.XPATH, ".//h2[contains(@class, 'CarVariantCard_variantName')]").text.strip()
                        if card_name == variant_name:
                            has_badge = len(card.find_elements(By.CLASS_NAME, "CarVariantCard_discontinuedBadge__bqsA8")) > 0
                            if not has_badge:
                                target_card = card
                                break
                    except:
                        continue

                if not target_card:
                    continue

                # Click the button
                select_button = target_card.find_element(By.XPATH, ".//button[contains(@class, 'CarVariantCard_exploreButton')]")
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", select_button)
                time.sleep(0.5)
                driver.execute_script("arguments[0].click();", select_button)
                time.sleep(4)

                # Extract specs
                variant_info = {
                    'name': variant_name,
                    'on_road_price': None,
                    'fuel_type': None,
                    'engine_capacity': None,
                    'mileage': None,
                    'seating_capacity': None,
                    'transmission': None
                }

                # On-road price
                try:
                    price_element = driver.find_element(By.CLASS_NAME, "MMVPriceBreakup_priceValue__uoYlB")
                    variant_info['on_road_price'] = price_element.text.strip()
                except:
                    pass

                # Fuel type
                try:
                    fuel_card = driver.find_element(By.XPATH, "//div[contains(@class, 'sf-content-block__card--fuel-type')]")
                    variant_info['fuel_type'] = fuel_card.find_element(By.CLASS_NAME, "SpecsAndFeatureCard_card__text__wxvgC").text.strip()
                except:
                    pass

                # Engine capacity
                try:
                    engine_card = driver.find_element(By.XPATH, "//div[contains(@class, 'sf-content-block__card--engine-capacity')]")
                    variant_info['engine_capacity'] = engine_card.find_element(By.CLASS_NAME, "SpecsAndFeatureCard_card__text__wxvgC").text.strip()
                except:
                    pass

                # Transmission
                try:
                    trans_card = driver.find_element(By.XPATH, "//p[contains(@class, 'SpecsAndFeatureCard_card__title') and contains(text(), 'Transmission')]/ancestor::div[contains(@class, 'SpecsAndFeatureCard_card__X_t2v')]")
                    variant_info['transmission'] = trans_card.find_element(By.CLASS_NAME, "SpecsAndFeatureCard_card__text__wxvgC").text.strip()
                except:
                    pass

                # ARAI mileage
                try:
                    mileage_card = driver.find_element(By.XPATH, "//p[contains(@class, 'SpecsAndFeatureCard_card__title__dpB5f') and contains(text(), 'ARAI mileage')]/ancestor::div[contains(@class, 'SpecsAndFeatureCard_card__X_t2v')]")
                    variant_info['mileage'] = mileage_card.find_element(By.CLASS_NAME, "SpecsAndFeatureCard_card__text__wxvgC").text.strip()
                except:
                    pass

                # Seat capacity
                try:
                    seat_card = driver.find_element(By.XPATH, "//p[contains(@class, 'SpecsAndFeatureCard_card__title__dpB5f') and contains(text(), 'Seat capacity')]/ancestor::div[contains(@class, 'SpecsAndFeatureCard_card__X_t2v')]")
                    variant_info['seating_capacity'] = seat_card.find_element(By.CLASS_NAME, "SpecsAndFeatureCard_card__text__wxvgC").text.strip()
                except:
                    pass

                variants_data.append(variant_info)

                driver.back()
                time.sleep(2)

                # Re-click "View all variants"
                try:
                    button = driver.find_element(By.CLASS_NAME, "VariantWisePrice_viewMoreLink__CkpjG")
                    driver.execute_script("arguments[0].click();", button)
                    time.sleep(1)
                except:
                    pass

            except:
                try:
                    driver.back()
                    time.sleep(2)
                except:
                    break
                continue

        return variants_data

    except:
        return []

    finally:
        if driver:
            try:
                driver.quit()
            except:
                pass

In [9]:
rows = []

for car in cars_to_check:
    variants = get_complete_variants(car)

    if variants:
        for v in variants:
            rows.append({
                "Model": car,
                "Variant": v.get("name", "N/A"),
                "Price": v.get("on_road_price", "N/A"),
                "Fuel": v.get("fuel_type", "N/A"),
                "Engine": v.get("engine_capacity", "N/A"),
                "Transmission": v.get("transmission", "N/A"),
                "Mileage": v.get("mileage", "N/A"),
                "Seats": v.get("seating_capacity", "N/A"),
            })

    time.sleep(2)

df = pd.DataFrame(rows)
df.to_csv("car_variants.csv", index=False, encoding="utf-8")

print("Saved to car_variants.csv using pandas")

Saved to car_variants.csv using pandas


In [10]:
df = pd.read_csv("car_variants.csv")
df

,Model,Variant,Price,Fuel,Engine,Transmission,Mileage,Seats
0,Honda City,1.5 SV-R,"₹11,95,300",Petrol,1498 cc,Manual,17.0 kmpl,5 seater
1,Honda City,1.5 V-R,"₹12,69,500",Petrol,1498 cc,Manual,17.0 kmpl,5 seater
2,Honda City,1.5 VX-R,"₹13,72,900",Petrol,1498 cc,Manual,17.0 kmpl,5 seater
3,Honda City,1.5 V-R CVT,"₹13,90,200",Petrol,1498 cc,Automatic,18.0 kmpl,5 seater
4,Honda City,Elegant Edition CVT,"₹14,05,000",Petrol,1498 cc,Automatic,18.0 kmpl,5 seater
5,Honda City,1.5 Sports CVT,"₹14,37,500",Petrol,1498 cc,Automatic,18.0 kmpl,5 seater
6,Honda City,1.5 ZX-R,"₹14,86,800",Petrol,1498 cc,Manual,17.0 kmpl,5 seater
7,Honda City,1.5 VX-R CVT,"₹14,93,500",Petrol,1498 cc,Automatic,18.0 kmpl,5 seater
8,Honda City,1.5 ZX-R CVT,"₹16,07,400",Petrol,1498 cc,Automatic,18.0 kmpl,5 seater
9,Honda City,1.5 ZX-R eHEV,"₹19,48,200",Hybrid,1498 cc,Automatic,27.0 kmpl,5 seater


In [17]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Model         32 non-null     object
 1   Variant       32 non-null     object
 2   Price         31 non-null     object
 3   Fuel          31 non-null     object
 4   Engine        31 non-null     object
 5   Transmission  31 non-null     object
 6   Mileage       31 non-null     object
 7   Seats         31 non-null     object
dtypes: object(8)
memory usage: 2.1+ KB
None


In [18]:
print(df.isnull().sum())

Model           0
Variant         0
Price           1
Fuel            1
Engine          1
Transmission    1
Mileage         1
Seats           1
dtype: int64


In [19]:
df["Price"] = df["Price"].astype(str)

df["Price"] = (
    df["Price"]
    .str.replace("₹", "", regex=False)
    .str.replace(",", "", regex=False)
    .str.strip()
)

# Convert to float or int
df["Price"] = df["Price"].astype(float)


In [20]:
df["Engine"] = (
    df["Engine"]
    .astype(str)
    .str.replace("cc", "", regex=False)
    .str.replace("CC", "", regex=False)
    .str.replace("cc.", "", regex=False)
    .str.replace(" ", "", regex=False)
)


df["Engine"] = pd.to_numeric(df["Engine"], errors="coerce")

df["Engine"] = df["Engine"].fillna(0).astype(int)

In [21]:
df["Mileage"] = df["Mileage"].astype(str)
df["Mileage"] = (
    df["Mileage"]
    .str.replace("kmpl", "", regex=False)
    .str.replace("KMPL", "", regex=False)
    .str.replace("Kmpl", "", regex=False)
    .str.replace(" ", "", regex=False)   # remove spaces
)

df["Mileage"] = pd.to_numeric(df["Mileage"], errors="coerce")

df["Mileage"] = df["Mileage"].fillna(0)

In [22]:
df["Seats"] = df["Seats"].astype(str)

df["Seats"] = (
    df["Seats"]
    .str.replace("seater", "", regex=False)
    .str.replace("Seater", "", regex=False)
    .str.replace("SEATER", "", regex=False)
    .str.replace(" ", "", regex=False)
    .str.strip()
)

df["Seats"] = pd.to_numeric(df["Seats"], errors="coerce")

df["Seats"] = df["Seats"].fillna(0).astype(int)

In [23]:
df = df.apply(lambda col: col.str.strip() if col.dtype == "object" else col)
df

,Model,Variant,Price,Fuel,Engine,Transmission,Mileage,Seats
0,Honda City,1.5 SV-R,1195300.0,Petrol,1498,Manual,17.0,5
1,Honda City,1.5 V-R,1269500.0,Petrol,1498,Manual,17.0,5
2,Honda City,1.5 VX-R,1372900.0,Petrol,1498,Manual,17.0,5
3,Honda City,1.5 V-R CVT,1390200.0,Petrol,1498,Automatic,18.0,5
4,Honda City,Elegant Edition CVT,1405000.0,Petrol,1498,Automatic,18.0,5
5,Honda City,1.5 Sports CVT,1437500.0,Petrol,1498,Automatic,18.0,5
6,Honda City,1.5 ZX-R,1486800.0,Petrol,1498,Manual,17.0,5
7,Honda City,1.5 VX-R CVT,1493500.0,Petrol,1498,Automatic,18.0,5
8,Honda City,1.5 ZX-R CVT,1607400.0,Petrol,1498,Automatic,18.0,5
9,Honda City,1.5 ZX-R eHEV,1948200.0,Hybrid,1498,Automatic,27.0,5


from matplotlib import pyplot as plt
df['Price'].plot(kind='hist', bins=20, title='Price')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['Engine'].plot(kind='hist', bins=20, title='Engine')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['Mileage'].plot(kind='hist', bins=20, title='Mileage')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['Seats'].plot(kind='hist', bins=20, title='Seats')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(df['Fuel'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(df, x='Price', y='Fuel', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(df['Transmission'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(df, x='Price', y='Transmission', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(df['Model'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(df, x='Engine', y='Model', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)